# TransGNN_DTA 药物-靶点亲和力预测模型使用指南
> 基于Transformer与GNN的药物-靶点亲和力预测模型，支持多数据集训练与预测。  
> 作者：Quietpeng（[GitHub](https://github.com/Quietpeng)），相关论文已发表（如需引用请参考项目README）。

### 一、环境配置与项目初始化
#### 1.1 克隆项目仓库

In [ ]:
# 克隆仓库（已克隆可忽略）
!git clone https://github.com/Quietpeng/TransGNN_DTA.git
%cd TransGNN_DTA

#### 1.2 创建并激活虚拟环境（推荐最佳实践）

In [ ]:
# 创建Python虚拟环境（基于Python 3.12）
!python -m venv transgnn_env
# 激活环境（Linux/macOS）
!source transgnn_env/bin/activate
# Windows环境请使用：.\transgnn_env\Scripts\activate

#### 1.3 安装依赖包

In [ ]:
# 安装项目依赖（需提前根据硬件配置安装PyTorch与CUDA）
!pip install -r requirements.txt
# 可选：安装后台管理工具（用于远程训练监控）
!sudo apt install screen -y && sudo apt update

### 二、模型训练流程
#### 2.1 配置说明
**命令行配置信息**
| 参数名称       | 类型    | 默认值   | 说明                                                                 |
|----------------|---------|----------|----------------------------------------------------------------------|
| `b`    | int     | 32       | 训练批次大小，建议根据GPU内存调整（如32GB GPU可尝试64）             |
| `epochs`       | int     | 200      | 最大训练轮次，结合早停机制使用                                       |
| `dataset`      | str     | `raw_davis` | 数据集选择，支持`raw_davis`/`raw_kiba`/`benchmark_davis`等          |
| `lr`           | float   | 5e-4     | 初始学习率，配合AdamW优化器与学习率调度器动态调整                    |
| `model_config` | str     | `config.json` | 模型结构配置文件路径，包含嵌入维度、序列最大长度等关键参数          |

**超参数配置文件路径**：`config.json`（需提前根据数据集调整`drug_max_seq`与`target_max_seq`）

#### 2.2 快速启动训练（推荐后台运行）

In [ ]:
# 方式1：默认参数启动（结果输出至result.log）
!python train_reg.py &> result.log 

In [ ]:
# 方式2：指定参数启动（示例：使用raw_kiba数据集，批次大小128）
!python train_reg.py --dataset raw_kiba --batchsize 128 --lr 1e-4 &> result.log 

**注意事项**：
- 训练建议使用GPU（如32GB vGPU），笔记本电脑可能因资源不足导致崩溃
- 使用`screen`工具后台运行：

In [ ]:
!screen -S transgnn_train  # 创建新会话
  # 执行训练命令后按Ctrl+A+D退出会话
!screen -r transgnn_train   # 恢复会话

#### 2.3 训练监控与可视化
**可视化地址**：http://localhost:6006（本地）或服务器公网IP:6006（需开放防火墙）

In [ ]:
# 启动TensorBoard监控（默认端口6006，需提前安装screen）
!screen -dmS tensorboard bash -c 'tensorboard --logdir=log --host=0.0.0.0'
# 远程访问需配置端口转发：ssh -L 6006:localhost:6006 user@server_ip

### 三、模型预测流程
#### 3.1 加载预训练模型与配置

In [ ]:
import torch
import json
from preprocess import drug_encoder, target_encoder
from double_towers import TransGNNModel

# 示例数据（药物SMILES与蛋白质序列）
DRUG_EXAMPLE = "CC1=C(C=C(C=C1)NC2=NC=CC(=N2)N(C)C3=CC4=NN(C(=C4C=C3)C)C)S(=O)(=O)Cl"
PROTEIN_EXAMPLE = "MRGARGAWDFLCVLLLLLRVQTGSSQPSVSPGEPSPPSIHPGKSDLIVRVGDEIRLLCTDP"

# 配置文件与模型路径
MODEL_CONFIG_PATH = "config.json"
MODEL_PATH = "./models/DAVIS_bestCI_model_reg1.pth"  # 替换为实际训练好的模型路径

#### 3.2 模型初始化与设备配置

In [ ]:
# 加载模型配置
model_config = json.load(open(MODEL_CONFIG_PATH, 'r'))
# 初始化模型
model = TransGNNModel(model_config)
# 检查是否有可用的 GPU
use_cuda = torch.cuda.is_available()
device = torch.device('cuda:0' if use_cuda else 'cpu')
model = model.to(device)

# 加载训练好的模型权重
try:
    checkpoint = torch.load(MODEL_PATH, map_location=device, weights_only=True)
    # 调整 decoder 层的输入维度
    model.decoder[0] = nn.Linear(list(checkpoint['decoder.0.weight'].shape)[1], model.decoder[0].out_features).to(device)
    model.load_state_dict(checkpoint)
    print("Successfully loaded model state from checkpoint.")
except Exception as e:
    print(f"Error loading checkpoint: {e}")

model.eval()

#### 3.3 数据预处理与预测

In [ ]:
# 序列编码（返回特征向量与掩码）
d_out, mask_d_out = drug_encoder(DRUG_EXAMPLE)
t_out, mask_t_out = target_encoder(PROTEIN_EXAMPLE)

# 转换为张量并移动至设备
d_tensor = torch.LongTensor(d_out).unsqueeze(0).to(device)
mask_d_tensor = torch.LongTensor(mask_d_out).unsqueeze(0).to(device)
t_tensor = torch.LongTensor(t_out).unsqueeze(0).to(device)
mask_t_tensor = torch.LongTensor(mask_t_out).unsqueeze(0).to(device)

# 执行预测
with torch.no_grad():
    prediction = model(d_tensor, t_tensor, mask_d_tensor, mask_t_tensor).cpu().numpy()

print(f"亲和力预测值：{prediction[0][0]:.4f}")  # 输出格式化为4位小数

### 四、高级功能说明
#### 4.1 早停机制与邮件通知
- **早停配置**：在`config.json`中设置`early_stopping_patience`（默认20轮）
- **邮件通知**：启用邮箱配置后，训练完成/失败时自动发送通知  
  ```json
  "email": {
    "enabled": true,
    "sender_email": "your_email@example.com",
    "sender_password": "授权码",
    "receiver_email": "recipient@example.com",
    "smtp_server": "smtp.qq.com",  # 以QQ邮箱为例
    "smtp_port": 465
  }
  ```

#### 4.2 多卡训练支持
如需使用多GPU训练，修改`train_reg.py`中数据加载部分：  
```python
# 添加 DistributedDataParallel 支持（示例）
if torch.cuda.device_count() > 1:
    model = torch.nn.DataParallel(model)
```

### 五、引用建议
若使用本模型进行研究，请在论文中引用 
<!-- ```bibtex
@article{quietpeng2023transgnn,
  title={TransGNN: A Hybrid Transformer-GNN Architecture for Drug-Target Affinity Prediction},
  author={Quietpeng},
  journal={Journal of Computational Biology},
  year={2023},
  volume={30},
  number={5},
  pages={891-903}
}
``` -->